In [ ]:
# 1. 加载要索引的数据。  
# 使用WebBaseLoader加载网页数据  
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

# 2. 文档分片，并通过向量存储构建文档索引   
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
# FAISS：轻量化的本地存储  
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

#文档分片  
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# 分片数据添加索引  
vector = FAISS.from_documents(documents, embeddings)

# 索引转为检索器 
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()

In [ ]:

from langchain.tools.retriever import create_retriever_tool

# 封装成工具
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "搜索与LangSmith相关的信息。有关LangSmith的任何问题，您必须使用此工具！",
)

# 
tools = [retriever_tool]

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# 加载官方提示
prompt = hub.pull("hwchase17/openai-functions-agent")

# 您需要设置OPENAI_API_KEY环境变量，或者将其作为参数`api_key`传递。
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "langsmith如何帮助测试？"})

In [ ]:
agent_executor.invoke({"input": "天气如何"})

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="LangSmith可以帮助测试我的LLM应用程序吗？"), AIMessage(content="可以！")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "告诉我如何进行"
})